In [1]:
%%capture
!pip install pygame
!pip install pygad

!git clone https://github.com/karinemiras/evoman_framework.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [2]:
import sys, os
import numpy as np
import pandas as pd

import random

from evoman.environment import Environment
from evoman.controller import Controller
from demo_controller import player_controller
from deap import base, creator, tools, algorithms
import multiprocessing
import optuna

import seaborn as sns
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
def parameter_count(hidden_neurons):
    if hidden_neurons>0:
        n_w = (20*hidden_neurons) + (hidden_neurons*5)
        n_b = hidden_neurons + 5
        return n_w + n_b
    else:
        return (20*5)+5
    
def fitness_function(solution):
    sum_gain = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        gain = res[1]-res[2]
        sum_gain += gain
    return (sum_gain,)

"""
def fitness_function(solution):
    defeated = 0
    for enemy in range(1,9):
        env.update_parameter("enemies",[enemy])
        res = env.play(np.array(solution))
        if res[2]<=0:
            defeated += 1
    return (defeated,)
"""

'\ndef fitness_function(solution):\n    defeated = 0\n    for enemy in range(1,9):\n        env.update_parameter("enemies",[enemy])\n        res = env.play(np.array(solution))\n        if res[2]<=0:\n            defeated += 1\n    return (defeated,)\n'

In [4]:
def cxWholeAritmetic(parent1,parent2,alpha):
    child1 = (alpha*np.array(parent1))+((1-alpha)*np.array(parent2))
    child2 = (alpha*np.array(parent2))+((1-alpha)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

def cxBlend(parent1,parent2,alpha):
    gamma = (1+2*alpha)*np.random.uniform(size=(len(parent1,))) - alpha
    child1 = (gamma*np.array(parent1))+((1-gamma)*np.array(parent2))
    child2 = (gamma*np.array(parent2))+((1-gamma)*np.array(parent1))
    return creator.Individual(child1),creator.Individual(child2)

In [5]:
def mutGaussian(ind,sigma,indpb):
    mutation = [np.random.normal(0,sigma) if np.random.uniform()<indpb else 0 for i in range(len(ind))]
    return creator.Individual(np.array(ind) + np.array(mutation))

def mutGaussianAdaptive(ind,sigma,tau,indpb):
    new_sigma = sigma * np.exp(np.random.normal(0,tau,size=(len(ind),)))
    new_sigma[new_sigma<0.1] = 0.1
    mutation = [np.random.normal(0,s) if np.random.uniform()<indpb else 0 for s in new_sigma]
    return creator.Individual(np.array(ind) + np.array(mutation))

In [6]:
def genetic_algorithm(initial_range,pop_size,n_offspring,tournsize_parent,cxpb,cx_type,
                      alpha,mutpb,mut_type,sigma,indpb,tau,tournsize_survival,ngen,verbose):
    
    global individual_size,env,creator,stats
    
    toolbox = base.Toolbox()
    toolbox.register("weight_bin", np.random.uniform,-initial_range,initial_range)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", fitness_function)

    pop = toolbox.population(n=pop_size)
    hof = {'best_solution':-np.inf,'max_value':-np.inf}

    with multiprocessing.Pool() as pool:
        # evaluate initial population
        for ind, fit in zip(pop, pool.map(toolbox.evaluate,pop)):
            ind.fitness.values = fit

        # record best solution
        for ind in pop:
            if ind.fitness.values[0]>hof['max_value']:
                hof['best_solution'] = ind
                hof['max_value'] = ind.fitness.values[0]

        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals'] + (stats.fields if stats else []) + ['max_hof']
        record = stats.compile(pop) if stats else {}

        logbook.record(gen=0, nevals=len(pop), **record, max_hof=hof['max_value'])
        if verbose==1:
            print(logbook.stream)

        for g in range(ngen):
            # select parents
            parents = tools.selTournament(individuals=pop,k=n_offspring*pop_size,tournsize=tournsize_parent)

            # create offspring
            offspring = []
            for child1, child2 in zip(parents[::2], parents[1::2]):
                if np.random.uniform() < cxpb:
                    if cx_type=="whole_aritmetic":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                        else:
                            child1,child2 = cxWholeAritmetic(child1, child2, alpha)
                    elif cx_type=="blend":
                        if alpha=="random":
                            alpha=np.random.uniform()
                            child1,child2 = cxBlend(child1, child2, alpha)
                        else:
                            child1,child2 = cxBlend(child1, child2, alpha)
                offspring.append(child1)
                offspring.append(child2)

            # apply mutation to offspring
            for i in range(len(offspring)):
                if np.random.uniform() < mutpb:
                    if mut_type=="normal":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                        else:
                            offspring[i] = mutGaussian(offspring[i],sigma,indpb)
                    elif mut_type=="adaptive":
                        if indpb=="random":
                            indpb=np.random.uniform()
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)
                        else:
                            offspring[i] = mutGaussianAdaptive(offspring[i],sigma,indpb,tau)       

            # evaluate offspring
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            for ind, fit in zip(invalid_ind, pool.map(toolbox.evaluate,invalid_ind)):
                    ind.fitness.values = fit

            # record best solution
            for ind in offspring:
                if ind.fitness.values[0]>hof['max_value']:
                    hof['best_solution'] = ind
                    hof['max_value'] = ind.fitness.values[0]

            # select the next generation of the population
            pop = tools.selTournament(individuals=offspring,k=pop_size,tournsize=tournsize_survival)

            record = stats.compile(pop) if stats else {}
            logbook.record(gen=g+1, nevals=len(invalid_ind), **record, max_hof=hof['max_value'])
            if verbose==1:
                print(logbook.stream)
        pool.close()
        pool.join()

    return hof,logbook

In [7]:
hidden_neurons = 10
individual_size = parameter_count(hidden_neurons)

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
                  player_controller=player_controller(hidden_neurons),
                  speed="fastest",
                  enemymode="static",
                  level=2,
                  visuals=False,
                  logs="off")

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Std", np.std)

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5181:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5704:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default


In [8]:
def objective(trial):
    
    params = {'initial_range':trial.suggest_int("initial_range", 1,3,step=1),
              'pop_size':trial.suggest_int("pop_size", 50,50,step=50),
              'n_offspring':trial.suggest_int("n_offspring", 1,5,step=1),
              'tournsize_parent':trial.suggest_int("tournsize_parent", 1,10,step=1),
              'cxpb':trial.suggest_float("cxpb", 0.1, 1., step=0.1),
              'cx_type':trial.suggest_categorical("cx_type",["whole_aritmetic","blend"]),
              'alpha':trial.suggest_categorical("alpha",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'mutpb':trial.suggest_float("mutpb", 0.1, 1., step=0.1),
              'mut_type':trial.suggest_categorical("mut_type",["normal","adaptive"]),
              'sigma':trial.suggest_float("sigma", 0.5, 3, step=0.5),
              'indpb':trial.suggest_categorical("indpb",np.arange(0.1,1.1,0.1).tolist()+["random"]),
              'tau':trial.suggest_float("tau", 0.1, 1., step=0.1),
              'tournsize_survival':trial.suggest_int("tournsize_survival", 1,10,step=1),
              'ngen':trial.suggest_int("ngen", 30,30,step=20),
              'verbose':trial.suggest_int("verbose", 1,1,step=1)}
    
    global hof_tuning
    iteration_logs = []
    for i in range(3):
        hof,logs = genetic_algorithm(**params)
        logs_pd = pd.DataFrame(logs)
        iteration_logs.append(logs_pd)
        hof_tuning.append(hof)
    
    return pd.concat(iteration_logs)['Max'].mean()


In [9]:
hof_tuning = []

study = optuna.create_study(study_name="GA", direction="maximize")

study.optimize(objective, n_trials=500)

[I 2023-10-15 00:08:18,236] A new study created in memory with name: GA


gen	nevals	Mean   	Max   	Std    	max_hof
0  	50    	-641.52	-225.4	170.223	-225.4 
1  	104   	-258.98	-185.4	68.7778	-185.4 
2  	108   	-203.564	-82.8 	63.8921	-82.8  
3  	101   	-127.804	-82.8 	50.606 	-82.8  
4  	112   	-82.8   	-82.8 	0      	-82.8  
5  	104   	-92.94  	-77.6 	45.9764	-77.6  
6  	114   	-83.012 	-30.4 	30.3831	-30.4  
7  	102   	-57.964 	-30.4 	52.3509	-30.4  
8  	110   	-30.4   	-30.4 	3.55271e-15	-30.4  
9  	102   	-30.4   	-30.4 	3.55271e-15	-30.4  
10 	108   	-30.4   	-30.4 	3.55271e-15	-30.4  
11 	117   	-30.4   	-30.4 	3.55271e-15	-30.4  
12 	113   	-30.4   	-30.4 	3.55271e-15	-30.4  
13 	111   	-30.4   	-30.4 	3.55271e-15	-30.4  
14 	93    	-30.4   	-30.4 	3.55271e-15	-30.4  
15 	109   	-34.28  	-30.4 	27.16      	-30.4  
16 	118   	-33.288 	-30.4 	20.216     	-30.4  
17 	99    	-41.54  	-30.4 	52.9941    	-30.4  
18 	98    	-30.4   	-30.4 	3.55271e-15	-30.4  
19 	101   	-30.4   	-30.4 	3.55271e-15	-30.4  
20 	97    	-30.4   	-30.4 	3.55271e-15	-30.4  
21 	1

[I 2023-10-15 00:23:01,456] Trial 0 finished with value: -49.90107526881659 and parameters: {'initial_range': 1, 'pop_size': 50, 'n_offspring': 3, 'tournsize_parent': 8, 'cxpb': 0.5, 'cx_type': 'blend', 'alpha': 0.2, 'mutpb': 0.4, 'mut_type': 'normal', 'sigma': 1.0, 'indpb': 0.5, 'tau': 0.1, 'tournsize_survival': 5, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: -49.90107526881659.


30 	101   	126.8   	126.8 	2.84217e-14	126.8  
gen	nevals	Mean    	Max   	Std    	max_hof
0  	50    	-677.768	-225.4	138.598	-225.4 
1  	46    	-597.128	-211.2	200.614	-211.2 
2  	40    	-462.208	-211.2	208.588	-211.2 
3  	43    	-328.052	-211.2	212.598	-211.2 
4  	42    	-232.576	-211.2	104.721	-211.2 
5  	31    	-241.316	-211.2	120.819	-211.2 
6  	44    	-265.892	-211.2	164.432	-211.2 
7  	50    	-241.6  	-211.2	106.193	-211.2 
8  	38    	-242.728	-211.2	124.925	-211.2 
9  	40    	-242.128	-211.2	125.025	-211.2 
10 	43    	-301.52 	-211.2	207.401	-211.2 
11 	42    	-243.144	-211.2	96.4072	-211.2 
12 	47    	-260.336	-211.2	149.922	-211.2 
13 	41    	-257.104	-211.2	155.841	-211.2 
14 	43    	-245.856	-211.2	137.172	-211.2 
15 	40    	-233.464	-211.2	109.261	-211.2 
16 	40    	-245.12 	-140.8	140.534	-140.8 
17 	34    	-311.568	-140.8	210.176	-140.8 
18 	38    	-356.192	-140.8	256.278	-140.8 
19 	40    	-167.072	-140.8	100.319	-140.8 
20 	41    	-147.584	-140.8	47.488 	-140.8 
21 	41 

[I 2023-10-15 00:28:27,537] Trial 1 finished with value: -190.86881720430057 and parameters: {'initial_range': 1, 'pop_size': 50, 'n_offspring': 1, 'tournsize_parent': 10, 'cxpb': 0.8, 'cx_type': 'blend', 'alpha': 0.5, 'mutpb': 0.1, 'mut_type': 'adaptive', 'sigma': 2.0, 'indpb': 0.2, 'tau': 0.9, 'tournsize_survival': 1, 'ngen': 30, 'verbose': 1}. Best is trial 0 with value: -49.90107526881659.


...

30 	78    	-31.776 	-2.4 	5.99635    	-2.4   
gen	nevals	Mean    	Max   	Std    	max_hof
0  	50    	-646.296	-195.4	156.251	-195.4 
1  	184   	-234.596	-83   	93.8893	-83    
2  	186   	-89.744 	-69.4 	23.5592	-69.4  
3  	185   	-90.116 	-64.4 	35.9344	-64.4  
4  	181   	-79.7   	-45.6 	37.3451	-45.6  
5  	181   	-56.94  	-40.4 	13.7801	-40.4  
6  	180   	-45.404 	-16.4 	19.7028	-16.4  
7  	187   	-41.096 	-16.4 	20.1933	-16.4  
8  	181   	-19.048 	7.6   	11.1737	7.6    
9  	188   	-18.988 	7.6   	32.8155	7.6    
10 	182   	4.488   	14    	13.6067	14     
11 	185   	5.816   	25.8  	24.4818	25.8   
12 	178   	7.232   	25.8  	41.5997	25.8   
13 	190   	19.728  	25.8  	24.719 	25.8   
14 	181   	25.784  	62.2  	12.7019	62.2   
15 	178   	39.592  	62.2  	38.4833	62.2   
16 	185   	53.74   	62.2  	35.005 	62.2   
17 	184   	53.356  	62.2  	36.0355	62.2   
18 	189   	60.212  	62.2  	13.916 	62.2   
19 	174   	56.256  	62.2  	41.608 	62.2   
20 	184   	62.2    	62.2  	0      	62.2   
21 	186 

[I 2023-10-15 10:31:57,954] Trial 31 finished with value: 26.28602150537706 and parameters: {'initial_range': 3, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 8, 'cxpb': 0.7000000000000001, 'cx_type': 'blend', 'alpha': 1.0, 'mutpb': 0.7000000000000001, 'mut_type': 'adaptive', 'sigma': 1.0, 'indpb': 0.9, 'tau': 0.4, 'tournsize_survival': 9, 'ngen': 30, 'verbose': 1}. Best is trial 16 with value: 30.800000000000495.


30 	185   	97.62   	112.4 	58.8485    	112.4  
gen	nevals	Mean    	Max   	Std    	max_hof
0  	50    	-682.064	-337.4	129.148	-337.4 
1  	192   	-302.984	-102.8	91.2366	-102.8 
2  	196   	-205.748	-102.8	63.8387	-102.8 
3  	193   	-187.528	-102.8	70.8805	-102.8 
4  	197   	-167.532	-83.8 	77.2244	-83.8  
5  	192   	-158.096	-83.8 	73.7685	-83.8  
6  	196   	-157.4  	-49.6 	71.0565	-49.6  
7  	194   	-179.364	-83.8 	93.1223	-49.6  
8  	191   	-118.084	-83.8 	56.9707	-49.6  
9  	192   	-135.38 	-83.8 	90.1842	-49.6  
10 	195   	-144.228	-83.8 	67.0517	-49.6  
11 	197   	-138.028	-83.8 	73.6122	-49.6  
12 	193   	-118.384	-83.8 	56.5636	-49.6  
13 	199   	-158.852	-83.8 	88.2232	-49.6  
14 	198   	-162.852	-80.8 	85.0817	-49.6  
15 	195   	-146.512	-66.8 	77.7875	-49.6  
16 	200   	-144.376	-26.6 	80.0798	-26.6  
17 	193   	-111.808	-22   	73.8162	-22    
18 	192   	-94.864 	-22   	70.7837	-22    
19 	193   	-77.576 	-22   	64.4744	-22    
20 	193   	-85.288 	-22   	76.814 	-22    
21 	196

Process ForkPoolWorker-389:
Process ForkPoolWorker-391:
Process ForkPoolWorker-390:
Process ForkPoolWorker-392:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    resu

KeyboardInterrupt: 

In [ ]:
#Trial 16 finished with value: 30.800000000000495 and parameters: {'initial_range': 2, 'pop_size': 50, 'n_offspring': 4, 'tournsize_parent': 6, 'cxpb': 0.6, 'cx_type': 'blend', 'alpha': 0.8, 'mutpb': 0.9, 'mut_type': 'adaptive', 'sigma': 3.0, 'indpb': 0.8, 'tau': 0.30000000000000004, 'tournsize_survival': 7, 'ngen': 30, 'verbose': 1}

In [13]:
res_pd = pd.DataFrame(hof_tuning).sort_values('max_value',ascending=False).reset_index()


In [15]:
res_pd.to_csv("v2_res.csv")

In [ ]:
"""
params = {'initial_range':,
          'pop_size':,
          'n_offspring':,
          'tournsize_parent':,
          'cxpb':,
          'cx_type':,
          'alpha':,
          'mutpb':,
          'mut_type':,
          'sigma':,
          'indpb':,
          'tau':,
          'tournsize_survival':,
          'ngen':,
          'verbose':}
""""